# "HR has requested some help with a project," my team lead, Jordan, explained.

"They have data from a survey given to a bunch of managers, and they'd like you to go through it to find:
* the average salary for a software engineer for each currency,
* the average salary for a software engineer for each currency _grouped by age_, and
* a comparison of the four currencies which are most common in the data.

"And just a heads up: the data is a bit messy, since there are some free-response text fields in the survey, so it will need some cleaning. You'll also need to grab currency conversions to compare the salaries."

"I'm on it!" I replied, and headed back to my desk to get started...

## My tasks
* Explore the dataset, handling missing entries
* Determine the salaries for software developers and engineers in USD
* Determine the average S/E salary for each currency and the average S/E salary for each currency based on age
* Visualize a comparison by plotting the salaries based on age for the top 4 currencies in the merged dataset

## Dependencies

In [10]:
# !conda install -c conda-forge thefuzz -y

In [11]:
# !conda install python-Levenshtein -y

In [5]:
# import Levenshtein
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
# from thefuzz import fuzz, process

## Read in and explore the data

* I like to glance through the first few rows of the dataframe to get an idea for what I'm dealing with

In [9]:
df = pd.read_csv("../data301_proj1/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.csv")
df.head()

,Timestamp,How old are you?,Industry,Job title,Additional context on job title,Annual salary,Other monetary comp,Currency,Currency - other,Additional context on income,Country,State,City,Overall years of professional experience,Years of experience in field,Highest level of education completed,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


* Not all the columns in the dataset are pertinent to the questions raised, so we'll filter out some.

In [13]:
pertinent_cols = ['Job title', "How old are you?",
       'Additional context on job title', 'Annual salary',
       'Other monetary comp', 'Currency', 'Currency - other',
       'Additional context on income', 'Country', 'State', 'City']
df_pert = df[pertinent_cols]

,Job title,How old are you?,Additional context on job title,Annual salary,Other monetary comp,Currency,Currency - other,Additional context on income,Country,State,City
0,Research and Instruction Librarian,25-34,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston
1,Change & Internal Communications Manager,25-34,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge
2,Marketing Specialist,25-34,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga
3,Program Manager,25-34,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee
4,Accounting Manager,25-34,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville
...,...,...,...,...,...,...,...,...,...,...,...
27604,Training and Development Coordinator,25-34,NaN,41600,2000.0,USD,NaN,NaN,United States,Kentucky,Louisville
27605,contract manager,25-34,NaN,80000,6000.0,CAD,NaN,NaN,canada,NaN,toronto
27606,RPA Developer,25-34,NaN,75000,0.0,USD,NaN,NaN,USA,Texas,Houston
27607,IT Specialist,25-34,Desktop Support/Healp Desk,45760,0.0,USD,NaN,NaN,USA,Wisconsin,Oconomowoc


* A quick way to reduce cardinality of the 'Job title' text field is to lowercase all values

In [15]:
df_pert['Job title'] = df_pert['Job title'].map(lambda x: x.lower())

* Since this is a time-sensitive request, we'll create a simple Boolean flag with a rough filter of a few ways that people seem to have indicated their title. This will miss typos ('sotfware engineer' will be a 0) and allow non-software fields ('commercial real estate developer' will be a 1), but there's enough data and it captures enough of the actual tech roles to be a reasonable approximation. If precision was important, I would consider some regex and more complex filters.

In [17]:
df_pert["engineer_yn"] = df_pert["Job title"].apply(lambda x: 1 if "software" in x or "sw" in x or "developer" in x else 0)
df_pert["engineer_yn"].value_counts()

0    26014
1     1595
Name: engineer_yn, dtype: int64

In [18]:
df_pert.sort_values("engineer_yn", inplace=True)

In [19]:
df_pert.isnull().sum()

Job title                              0
How old are you?                       0
Additional context on job title    20463
Annual salary                          0
Other monetary comp                 7146
Currency                               0
Currency - other                   27427
Additional context on income       24603
Country                                0
State                               4911
City                                  75
engineer_yn                            0
dtype: int64

* Now I'll clean the 'Country' field, which unfortunately was a free text field (like 'Job title') instead of a dropdown list.

In [20]:
df_pert["Country"].value_counts()

United States      8844
USA                7847
US                 2572
Canada             1549
United States       652
                   ... 
europe                1
Can                   1
UNited States         1
Cayman Islands        1
the netherlands       1
Name: Country, Length: 364, dtype: int64

In [ ]:
df_pert["clean_country"] = df_pert["Country"].apply(lambda x: x.lower().strip())
df_pert["clean_country"].value_counts()

In [ ]:
# I know there are WAY more elegant ways to do this (regex, in particular, or using fuzzy string matching/Levenshtein distance),
# but I ran out of time, so I mostly brute-forced it...sorry!
df_pert["clean_country"] = df_pert["clean_country"].apply(lambda x: "usa" if x == "us" or re.search('unit.+ sta.+', x) or "usa" in x or "u.s" in x or "u. s" in x else x)
df_pert["clean_country"].value_counts()

In [ ]:
df_pert.columns

In [ ]:
df_pert = df_pert[['Job title', "How old are you?", 'Annual salary', 'Other monetary comp', 'Currency', 'Currency - other', 'Additional context on income', 'engineer_yn', 'clean_country']]
df_se_only = df_pert[df_pert['engineer_yn'] == 1]
df_se_only

In [ ]:
df_se_only["Annual salary"] = df_se_only["Annual salary"].apply(lambda x: float(re.sub(",", "", x)))
df_se_only

In [ ]:
df_se_only["Other monetary comp"] = df["Other monetary comp"].fillna(0)
df_se_only.info()

In [ ]:
df_se_only["total_comp"] = df_se_only["Annual salary"] + df_se_only["Other monetary comp"]
df_se_only

In [ ]:
df_se_only.drop(["Annual salary", "Other monetary comp"], axis=1, inplace=True)

In [ ]:
df_se_only["Currency"].value_counts()

In [ ]:
df_sek = df_se_only[df_se_only["Currency"] == "SEK"]
df_sek

### Currency conversion

In [ ]:
df_curr = pd.read_csv("data/currency_converter.csv", header=1, nrows=42)
df_curr

In [ ]:
df_curr = df_curr[["Currency", "7-Jan-22", "10-Jan-22", "11-Jan-22"]]
df_curr

In [ ]:
df_curr["Currency"]

In [ ]:
df_curr["Currency"] = df_curr["Currency"].apply(lambda x: "EUR" if "Euro" in str(x) else x)
df_curr["Currency"] = df_curr["Currency"].apply(lambda x: "USD" if "U.S." in str(x) else x)
df_curr["Currency"] = df_curr["Currency"].apply(lambda x: "CAD" if "Canadian" in str(x) else x)
df_curr["Currency"] = df_curr["Currency"].apply(lambda x: "GBP" if "U.K." in str(x) else x)
df_curr["Currency"] = df_curr["Currency"].apply(lambda x: "CHF" if "Swiss franc" in str(x) else x)
df_curr["Currency"] = df_curr["Currency"].apply(lambda x: "SEK" if "Swedish" in str(x) else x)
df_curr["Currency"] = df_curr["Currency"].apply(lambda x: "AUD/NZD" if "Australian" in str(x) or "Zealand" in str(x) else x)
df_curr

In [ ]:
df_curr_conv = df_curr[["Currency", "10-Jan-22"]]
df_curr_conv

In [ ]:
currencies = ["EUR", "USD", "CAD", "GBP", "CHF", "SEK", "AUD/NZD"]
df_app_curr = df_curr_conv[df_curr_conv["Currency"].isin(currencies)]
df_app_curr.reset_index()

In [ ]:
df_app_curr.drop([22], inplace=True)
df_app_curr

In [ ]:
df_app_curr.to_csv("./data/curr_conv.csv")

In [ ]:
df_curr_conv = pd.read_csv("./data/curr_conv.csv")

### Merge the S/E dataframe with the currency converter

In [ ]:
df_merged = df_se_only.merge(df_curr_conv, on="Currency")
df_merged

In [ ]:
df_merged_copy = df_merged.copy()
df_merged["total_comp_usd"] = round(df_merged_copy["total_comp"] / df_merged_copy["10-Jan-22"], 2)
df_merged

In [ ]:
df_final = df_merged[["Job title", "How old are you?", "Currency", "clean_country", "total_comp_usd"]]
df_final

In [ ]:
df_final.sort_values("total_comp_usd", inplace=True)
df_final.reset_index()

In [ ]:
df_final.columns

In [ ]:
df_final.to_csv("./data/clean_dataset.csv", index=False)

In [ ]:
df_final = pd.read_csv("./data/clean_dataset.csv")

### It looks like there are a few outliers on the low end...let's remove those for better analysis

In [ ]:
# TODO: remove outliers
df_final = df_final[df_final['total_comp_usd'] > 5000]

In [ ]:
df_final.head()

# Solutions!

### First we'll calculate the average compensation, grouped by currency

In [ ]:
means = df_final.groupby(["Currency"]).mean()
means.sort_values('total_comp_usd', ascending=False, inplace=True)

In [ ]:
means["total_comp_usd"] = means["total_comp_usd"].apply(lambda x: round(x, 2))
means

In [ ]:
means.to_csv('./solutions/avg_comp_by_currency.csv')

### Next we'll calculate the average compensation, grouped by currency and broken out by age range of the developer

In [ ]:
means_by_age = df_final.groupby(["Currency", 'How old are you?']).mean()
# means_by_age.head()
means_by_age.sort_values(['Currency', 'How old are you?'], ascending=True, inplace=True)
means_by_age

In [ ]:
means_by_age.to_csv('./solutions/avg_comp_by_currency_by_age.csv')

### Lastly, plot the salaries, grouped by age, for the top four currencies in the merged dataset

#### Note: I am interpreting "top four" as meaning "four most commonly represented," NOT "four with the highest average compensation"

In [ ]:
means_by_age.reset_index(inplace=True)
means_by_age

In [ ]:
# determine the top four currencies
df_final['Currency'].value_counts()

In [ ]:
four_currs = ['USD', 'GBP', 'CAD', 'EUR']
# from pandas docs: df1.loc[lambda df: df['A'] > 0, :]
top_four = means_by_age.loc[lambda df: df['Currency'].isin(four_currs), :]
top_four

In [ ]:
plt.figure(figsize=(20, 10), facecolor='#b2beb5')
plt.axes().set_facecolor('#b2beb5')
splot=sns.barplot(
    data=top_four,
    x="How old are you?",
    order=['under 18', '18-24', '25-34', '35-44', '45-54', '55-64', '65 or over'],
    y="total_comp_usd",
    hue="Currency",
    hue_order=['USD', 'CAD', 'EUR', 'GBP'],
    palette="colorblind"
)
plt.ylabel("Mean total compensation (USD)", size=16)
plt.xlabel("Age", size=16)
plt.title("Average software developer compensation by age and currency", size=20)
sns.despine()
for p in splot.patches:
    # print(f"Is p.get_height a nan? {pd.isna(p.get_height())}")
    if pd.isna(p.get_height()):
        continue
    else:
        splot.annotate(format(round(p.get_height()/1000), '.0f')+"K",
                       (p.get_x() + p.get_width() / 2., p.get_height()),
                       ha = 'center', va = 'center',
                       size=10,
                       color='white',
                       fontweight='bold',
                       xytext = (0, -12),
                       textcoords = 'offset points')
plt.legend(loc='upper left', fontsize=16, facecolor='#E5E4E2')
plt.savefig('./solutions/viz.png')